# Transferencia de Estilo
## Jorge Adrián Olmos Morales.
### jorgeadrianolmos@gmail.com

Empezaremos por importar las librerias necesarias
#### (nótese la importación de la libreria pladml.keras, la cuál es una libreria para dar soporte de aceleracion hardware a computadoras AMD) [Más info](https://github.com/plaidml/plaidml) 

In [ ]:
import plaidml.keras
plaidml.keras.install_backend()

import numpy as np
from PIL import Image
from keras import backend
from keras.applications.vgg16 import VGG16
from scipy.optimize import fmin_l_bfgs_b

A continuacion definimos los hyperparámetros 

In [ ]:
ITERATIONS = 20
CHANNELS = 3
IMAGE_WIDTH = 350 
IMAGE_HEIGHT = 300
IMAGENET_MEAN_RGB_VALUES = [123.68, 116.779, 103.939]
CONTENT_WEIGHT = .0005
STYLE_WEIGHT = 8
TOTAL_VARIATION_WEIGHT = 1
TOTAL_VARIATION_LOSS_FACTOR = 1

Definimos los paths para nuestros archivos de entrada y salida

In [ ]:
estilo = "path_to_foto_estilo" # modifica este path para concordar con el path de la imagen de la cual quieres tomar el estilo
foto = "path_to_foto_contenido"# modifica este path para concordar con el path de la imagen a la cual le quieres poner el estilo
input_image_path = "input.png"
style_input_path = "style.png"
output_image_path = "output.png"
combined_image_path = "combined.png"
content_image_path = foto+".jpeg" # modificar dependiendo del tipo de archivo de la imagen
style_image_path = estilo+".jpg"

Reajustamos las dimensiones de nuestras imágenes de entrada para que puedan ser procesadas por el modelo, que en este caso es la red neuronal VGG-16 (que por cierto es un modelo ya entrenado) 

In [ ]:
input_image = Image.open(content_image_path)
input_image = input_image.resize((IMAGE_WIDTH, IMAGE_HEIGHT), Image.LANCZOS)
input_image.save(input_image_path)

style_image = Image.open(style_image_path)
style_image = style_image.resize((IMAGE_WIDTH, IMAGE_HEIGHT), Image.LANCZOS)
style_image.save(style_input_path)


Creamos los arreglos multidimensionales que entraran en el modelo, normalizamos la información contenida en ellos y cambiamos los valores de RGB a BGR


In [ ]:
# Data normalization and reshaping from RGB to BGR
input_image_array = np.asarray(input_image, dtype="float32")
input_image_array = np.expand_dims(input_image_array, axis=0)
input_image_array[:, :, :, 0] -= IMAGENET_MEAN_RGB_VALUES[2]
input_image_array[:, :, :, 1] -= IMAGENET_MEAN_RGB_VALUES[1]
input_image_array[:, :, :, 2] -= IMAGENET_MEAN_RGB_VALUES[0]
input_image_array = input_image_array[:, :, :, ::-1]

style_image_array = np.asarray(style_image, dtype="float32")
style_image_array = np.expand_dims(style_image_array, axis=0)
style_image_array[:, :, :, 0] -= IMAGENET_MEAN_RGB_VALUES[2]
style_image_array[:, :, :, 1] -= IMAGENET_MEAN_RGB_VALUES[1]
style_image_array[:, :, :, 2] -= IMAGENET_MEAN_RGB_VALUES[0]
style_image_array = style_image_array[:, :, :, ::-1]

A continuación definimos las funciones para calcular la pérdida de contenido y la pérdida de estilo entre la imágen generada por el modelo y las imágenes de entrada. Son estas las funciones que trataremos de minimizar con el método de optimización L-BFGS. 

[Más info.](https://towardsdatascience.com/style-transfer-styling-images-with-convolutional-neural-networks-7d215b58f461) 

In [ ]:
def content_loss(content, combination):
    return backend.sum(backend.square(combination - content))

def gram_matrix(x):
    features = backend.batch_flatten(backend.permute_dimensions(x, (2, 0, 1)))
    gram = backend.dot(features, backend.transpose(features))
    return gram


def compute_style_loss(style, combination):
    style = gram_matrix(style)
    combination = gram_matrix(combination)
    size = IMAGE_HEIGHT * IMAGE_WIDTH
    return backend.sum(backend.square(style - combination)) / (4. * (CHANNELS ** 2) * (size ** 2))

def total_variation_loss(x):
    a = backend.square(x[:, :IMAGE_HEIGHT-1, :IMAGE_WIDTH-1, :] - x[:, 1:, :IMAGE_WIDTH-1, :])
    b = backend.square(x[:, :IMAGE_HEIGHT-1, :IMAGE_WIDTH-1, :] - x[:, :IMAGE_HEIGHT-1, 1:, :])
    return backend.sum(backend.pow(a + b, TOTAL_VARIATION_LOSS_FACTOR))

def evaluate_loss_and_gradients(x):
    x = x.reshape((1, IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS))
    outs = backend.function([combination_image], outputs)([x])
    loss = outs[0]
    gradients = outs[1].flatten().astype("float64")
    return loss, gradients

class Evaluator:

    def loss(self, x):
        loss, gradients = evaluate_loss_and_gradients(x)
        self._gradients = gradients
        return loss

    def gradients(self, x):
        return self._gradients

Combinamos nuestras imagenes de entrada en un tensor y alimentamos a la red con el.  

In [ ]:
input_image = backend.variable(input_image_array)
style_image = backend.variable(style_image_array)
combination_image = backend.placeholder((1, IMAGE_HEIGHT, IMAGE_WIDTH, 3))

input_tensor = backend.concatenate([input_image,style_image,combination_image], axis=0)
model = VGG16(input_tensor=input_tensor, include_top=False)

Recorremos el modelo y tomamos las activaciones generadas por el tensor de entrada en las capas deseadas (en este caso las que creemos representan el contenido y el estilo). Usaremos estas activaciones para ir calculando la pérdida global definida por "loss"

In [ ]:
layers = dict([(layer.name, layer.output) for layer in model.layers])

content_layer = "block2_conv2"
layer_features = layers[content_layer]
content_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]

loss = backend.variable(0.)
loss += CONTENT_WEIGHT * content_loss(content_image_features,
                                      combination_features)


style_layers = ["block1_conv2", "block2_conv2", "block3_conv3", "block4_conv3", "block5_conv3"]
for layer_name in style_layers:
    layer_features = layers[layer_name]
    style_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    style_loss = compute_style_loss(style_features, combination_features)
    loss += (STYLE_WEIGHT / len(style_layers)) * style_loss
    
loss += TOTAL_VARIATION_WEIGHT * total_variation_loss(combination_image)

outputs = [loss]
outputs += backend.gradients(loss, combination_image)


evaluator = Evaluator()

Creamos un tensor igual a la imagen a la cual le queremos poner el estilo. Este tensor es modificado por el algoritmo L-BFGS para tratar de minimizar la funcion de error que definimos. Esto será lo que le pondra el estilo deseado a nuestra imagen.    

In [ ]:
x = input_image_array

x, loss, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(), fprime=evaluator.gradients, maxiter=ITERATIONS, iprint=5)
print("Completed with loss %d" % (loss))

Guardamos el tensor modificado ya como imagen 

In [ ]:
x = x.reshape((IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS))
x = x[:, :, ::-1]
x[:, :, 0] += IMAGENET_MEAN_RGB_VALUES[2]
x[:, :, 1] += IMAGENET_MEAN_RGB_VALUES[1]
x[:, :, 2] += IMAGENET_MEAN_RGB_VALUES[0]
x = np.clip(x, 0, 255).astype("uint8")
output_image = Image.fromarray(x)
output_image.save(output_image_path)